<a href="https://colab.research.google.com/github/Adonai02/colab-/blob/main/English_Language_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers
!pip install torch
!pip install -q kaggle
!pip install scikit-multilearn

In [ ]:
import pandas as pd
import numpy as np
import sys, time, os, pytz, shutil, pickle
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, TrainingArguments, AutoModel, AutoConfig, Trainer, EarlyStoppingCallback, PrinterCallback
from google.colab import files
from skmultilearn.model_selection import iterative_train_test_split
from transformers.modeling_outputs import SequenceClassifierOutput

In [ ]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

IndexError: ignored

In [ ]:
os.listdir("/root/.kaggle/")
# !touch ~/.kaggle/kaggle.json

['kaggle.json']

In [ ]:
# ! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c feedback-prize-english-language-learning


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 169, in authenticate
    self._load_config(config_data)
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 210, in _load_config
    raise ValueError('Error: Missing %s in configuration.' % item)
ValueError: Error: Missing username in configuration.


In [ ]:
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c feedback-prize-english-language-learning
! unzip feedback-prize-english-language-learning.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open feedback-prize-english-language-learning.zip, feedback-prize-english-language-learning.zip.zip or feedback-prize-english-language-learning.zip.ZIP.


In [90]:
class CustomDataset(Dataset):

  def __init__(self, comentarios, labels, tokenizer, max_len):
      self.tokenizer = tokenizer
      self.comentario = comentarios
      self.targets = labels
      self.max_len = max_len

  def __len__(self):
      return len(self.comentario)

  def __getitem__(self, index):

      inputs = self.tokenizer.encode_plus(
          self.comentario[index],
          None,
          truncation = True,
          add_special_tokens = True,
          max_length = self.max_len,
          padding = "max_length",
          return_token_type_ids = False
      )
      
      output_dict = {
          'input_ids': torch.tensor(inputs['input_ids'], dtype = torch.long),
          'attention_mask': torch.tensor(inputs['attention_mask'], dtype = torch.long),
          'labels': torch.from_numpy(self.targets).type(dtype = torch.long)
      }

      return output_dict




class TextClassifier(torch.nn.Module):
    def __init__(self, model_path, num_classes, loss_fct, n_extra_layers = 0, tokenizer = None):
        super(TextClassifier, self).__init__()
        
        self.loss_fct = loss_fct
        
        self.pretrained_model =  AutoModel.from_pretrained(model_path)
        self.pretrained_model.config.output_attentions = False
        
        self.pretrained_model_dim = list(self.pretrained_model.named_parameters())[-1][1].shape[0]
        self.dropout = torch.nn.Dropout(0.1)
        self.num_classes = num_classes
        
        self.out_layer = torch.nn.Linear(self.pretrained_model_dim, num_classes, bias = True)
        # print(self.pretrained_model_dim, num_classes)
            
        self.extra_layers = torch.nn.ModuleList([torch.nn.Linear(self.pretrained_model_dim, self.pretrained_model_dim) for _ in range(n_extra_layers)])
        self.config = self.pretrained_model.config
                
        if tokenizer is not None:
            if tokenizer.pad_token_id is None:
                tokenizer.pad_token = tokenizer.eos_token
            
            self.config.pad_token_id = tokenizer.pad_token_id        
        else:
            self.config.pad_token_id = None

        self.config.pad_token_id = None if tokenizer is None else tokenizer.pad_token_id
        self.loss_fct = loss_fct
        
    def num_parameters(self):
        return np.sum(list(map(lambda x: np.prod(x.shape), self.parameters())))
        
    def forward(self, input_ids, attention_mask, labels = None):
        
        _, pooler_output = self.pretrained_model(input_ids, attention_mask = attention_mask, return_dict = False)
            
        x = self.dropout(pooler_output)

        for extra_fc_layer in self.extra_layers:
            x = extra_fc_layer(x)
            
        output = self.out_layer(x)  #(n_batches, n_clases)
        
        loss = None
        
        if labels is not None:            
            input_labels = labels.long().view(-1)            
            loss = self.loss_fct(output, input_labels)        

        return SequenceClassifierOutput(
            loss = loss,
            logits = output,
        )

class RMSELoss(torch.nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss


class MCRMSELoss(torch.nn.Module):
    def __init__(self, num_scored=3):
        super().__init__()
        self.rmse = RMSELoss()
        self.num_scored = num_scored

    def forward(self, yhat, y):
        score = 0
        for i in range(self.num_scored):
            score += self.rmse(yhat[:, i], y[:, i]) / self.num_scored

        return score

In [81]:
pwd

'/content'

In [ ]:
df_train = pd.read_csv("train.csv")

In [ ]:
df_train

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


In [ ]:
df_test = pd.read_csv("test.csv")

In [78]:
df_test

,text_id,full_text
0,0000C359D63E,when a person has no experience on a job their...
1,000BAD50D026,Do you think students would benefit from being...
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde..."


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
MAX_LEN = 512
test_size = 0.2
train_sampler = None
model_ckpt = "bert-base-cased"
model_dict = {"model_name" : "Language-Learning-v1", "model_path" : "bert-base-cased", "lang" : "english"}

In [ ]:
cols_values = df_train[df_train.columns[2:]].columns
n_labels = len(cols_values)

In [ ]:
x_train, y_train, x_val, y_val = iterative_train_test_split(
                                  df_train[["full_text"]].values, 
                                  df_train[cols_values].values, 
                                  test_size = test_size
                                 )

In [92]:
loss = MCRMSELoss(n_labels)

In [93]:
loss(torch.rand(3,6),torch.rand(3,6))

tensor(0.4523)

In [ ]:
batch_size=24
logging_steps = len(df_train["full_text"]) // batch_size
train_args = TrainingArguments(
    output_dir = "language_learning_finetune",
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy = 'epoch',
    logging_steps=logging_steps,
    fp16=True,
    load_best_model_at_end=True
)

Tokenize

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# class MultilabelTrainer(Trainer):
#     def __init__(self, loss_fct, num_labels, train_sampler = None, multiclass_model = False,  outputs=None, **kwargs):
#         super(MultilabelTrainer, self).__init__(**kwargs)
#         self.loss_fct = loss_fct
#         self.num_labels = num_labels
#         self.train_sampler = train_sampler
#         self.multiclass_model = multiclass_model
#         self.outputs = outputs
        
#     def compute_loss(self, model, inputs, return_outputs = False):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits
        
#         loss = self.loss_fct(logits.view(-1, self.num_labels), labels.long().view(-1))
        
#         return (loss, outputs) if return_outputs else loss

In [ ]:
train_dataset = CustomDataset(x_train.ravel(), y_train, tokenizer, MAX_LEN)
val_dataset = CustomDataset(x_val.ravel(), y_val, tokenizer, MAX_LEN)

In [ ]:
early_stopping_patience = 20
early_stopping_threshold = 0
cbks = [PrinterCallback()]
cbks.append(EarlyStoppingCallback(early_stopping_patience = early_stopping_patience, early_stopping_threshold=early_stopping_threshold))

In [ ]:
model = TextClassifier(model_dict["model_path"], n_labels, loss, tokenizer = tokenizer).to(device)

In [ ]:
Trainer(eva)

In [ ]:
trainer = Trainer(
    model = model,
    args = train_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = None,
    callbacks = cbks
)

Using cuda_amp half precision backend


In [ ]:
text = df_test.full_text.iloc[0]

In [ ]:
inputs = {k:v.to(device) for k,v in tokenizer.encode_plus(text, return_tensors="pt",truncation=True).items()}

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_output = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3129
  Num Epochs = 5
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 655
  Number of trainable parameters = 108314886


TypeError: ignored

In [ ]:
# Evalua
eval_dict = trainer.evaluate(eval_dataset=dataset)

# Find best thresholds    
predictions = trainer.predict(dataset)

Mon Nov  7 00:40:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cols_values = df_train[df_train.columns[2:]].values

In [ ]:
cols_values = df_train[df_train.columns[2:]].values
cols_tensors = torch.from_numpy(cols_values)